In [1]:
%config ZMQInteractiveShell.ast_node_interactivity = "all"
%pprint

Pretty printing has been turned OFF


In [2]:
import torch
import numpy as np
import torch.nn as nn
import sys
sys.path.append("../d2l_func/")
from sqdm import sqdm
from draw import set_fig_display
from decorate import cal_time

In [3]:
def loss_error_sample(list_length, sample_rate, equal_sample=True):
    """
    function: sample with list, return sample index(two choice: equal distance in sample and unequal distance)
    """
    if equal_sample:
        index = map(lambda x: int(x), list(np.linspace(0, list_length, int(list_length * sample_rate))))
    else:
        index_array = np.arange(list_length)
        index = np.random.choice(index_array, replace=False,
                                 size=int(list_length * sample_rate))
        index.sort()
    return list(index)

In [3]:
class TrainNet(object):
    def __init__(self, data_num, epoch_num, model, loss, train_iter, batch_size,
                optimizer=None, test_iter=None, evaluate=None, draw=False,
                draw_epoch=True, draw_mean=False, save_fig=False, save_path="./img/",
                accum_step=1, gpu=False, sample_rate=0, equal_sample=True):
        # params
        self.params = {
            "data_num": data_num,
            "epoch_num": epoch_num,
            'model': model,
            "loss": loss,
            "train_iter": train_iter,
            "batch_size": batch_size,
            "optimizer": optimizer,
            "test_iter": test_iter,
            "evaulate": evaluate,
            "draw": draw,
            "draw_epoch": draw_epoch,
            "draw_mean": draw_mean,
            "save_fig": save_fig,
            "save_path": save_path,
            "accum_step": accum_step,
            "gpu": gpu,
            "sample_rate": sample_rate,
            "equal_sample": equal_sample,
        }
        # storage loss and score in train and test
        self.train_loss_list, self.test_loss_list = [], []
        self.train_score_list, self.test_score_list = [], []
        # iteration num in each epoch
        self.iter_num = np.ceil(data_num / batch_size)
        # sample
        self.sample_index = loss_error_sample(data_num, sample_rate, equal_sample)
        
    @cal_time
    def train_epoch():
        for epoch in range(self.params["epoch_num"]):
            print(f"Epoch [{epoch + 1}/{self.params['epoch_num']}]")
            count, mean_train_loss, mean_train_score = 1., 0., 0.
            test_num, mean_test_loss, mean_test_score = 0., 0., 0.
            
            # load data
            for x, y in self.params["train_iter"]:
                # cuda available and want to use gpu
                if torch.cuda.is_available() and self.params["gpu"]:
                    # send tensor from cpu to gpu
                    x = x.cuda()
                    y = y.cuda()
                # train
                model.train()
                train_pred = self.params["model"](x)
                train_loss = self.params["loss"](train_pred, y) / accum_
                # calculate mean train loss
                mean_train_loss = (((count - 1) * mean_train_loss +
                                    accum_step * train_loss) / count).item()
                if draw_epoch:
                    if

SyntaxError: unexpected EOF while parsing (<ipython-input-3-b00a7b4e2558>, line 66)

In [ ]:
@cal_time
def train_epoch(data_num, epoch_num, model, loss, train_iter, batch_size,
                optimizer=None, test_iter=None, evaluate=None, draw=False,
                draw_mean=False, save_fig=False, save_path="./img/",
                accum_step=1, gpu=False):
    """
    function: training in pytorch (only with epoch), it will speed up in training in epoch
    params data_num: the number of sample in train set
    params epoch_num: the number of epoch
    params model: model which fit data
    params loss: calculate loss function
    params train_iter: train data loader
    params optimizer: torch optimizer which is used to update grad
    params test_iter: test data loader, use in testing
    params evaluate: criterion such as acc/f1 score
    params draw: draw figure with loss and score in train/test whether or not
    params draw_epoch: draw with data in iteration or epoch
    params save_fig: save figure whether or not
    params save_path: the path of saving figure
    params accum_step: use in gradient accumulation, after the number of accum_step,
                        it will be update the grad of parameters
    params gpu: if want to use gpu and cuda is available, it will send tensor to gpu
    """
    # training bar
    process_bar = sqdm()

    # init
    test_loss = test_score = "-"
    # storage loss and score in train and test
    train_loss_list, test_loss_list = [], []
    train_score_list, test_score_list = [], []
    # iteration num in each epoch
    iter_num = np.ceil(data_num / batch_size)

    for epoch in range(epoch_num):
        print(f"Epoch [{epoch + 1}/{epoch_num}]")
        count, mean_train_loss, mean_train_score = 1., 0., 0.
        test_num, mean_test_loss, mean_test_score = 0., 0., 0.
        for x, y in train_iter:
            # cuda available and want to use gpu
            if torch.cuda.is_available() and gpu:
                # send tensor from cpu to gpu
                x = x.cuda()
                y = y.cuda()
            # train
            # model.train()
            train_pred = model(x)
            train_loss = loss(train_pred, y) / accum_step
            # calculate mean train loss
            mean_train_loss = (((count - 1) * mean_train_loss +
                                accum_step * train_loss) / count).item()
            # when draw_mean is True, we save the mean_train_loss in each iteration,
            # otherwise, we save the train_loss in the last iteration in each epoch
            if count == iter_num:
                if draw_mean:
                    train_loss_list.append(mean_train_loss)
                else:
                    train_loss_list.append(accum_step * train_loss)
            # if parameter have criterion(evaluate), like accuracy/f1_score
            # use this criterion to calculate train_score
            if evaluate is not None:
                train_score = evaluate(x, y)
                mean_train_score = ((count - 1) * mean_train_score +
                                    train_score) / count
                # function like the draw_epoch in train loss
                if count == iter_num:
                    if draw_mean:
                        train_score_list.append(mean_train_score)
                    else:
                        train_score_list.append(train_score)

            # bp
            train_loss.backward()
            if (count % accum_step) == 0:
                # grad update
                optimizer.step()
                # clear grad
                optimizer.zero_grad()
            if count == iter_num:
                optimizer.zero_grad()

            # test loss
            with torch.no_grad():
                if (test_iter is not None) and (count == iter_num):
                    # eval model, it will stop dropout and batch normalization
                    model.eval()
                    for test_data, test_label in test_iter:
                        # calculate the num of test set
                        test_num += len(test_label)
                        # if cuda available and want to use gpu
                        if torch.cuda.is_available() and gpu:
                            test_data = test_data.cuda()
                            test_label = test_label.cuda()

                        test_pred = model(test_data)
                        test_loss = loss(test_pred, test_label).item()
                        mean_test_loss += len(test_label) * test_loss
                        # print(mean_test_loss)
                        # use this criterion to calculate test_score
                        if evaluate is not None:
                            test_score = evaluate(test_data, test_label)
                            mean_test_score += len(test_label) * test_score
                    # result
                    mean_test_loss /= test_num
                    test_loss_list.append(mean_test_loss)
                    if evaluate is not None:
                        mean_test_score /= test_num
                        test_score_list.append(mean_test_score)
                    # change to train model
                    model.train()

            # update counter
            count += 1
            # training bar
            if evaluate is None:
                mean_train_score = "-"
                mean_test_score = "-"
            if test_iter is None:
                mean_test_loss = "-"
                mean_test_score = "-"
            # use mean loss and score in each epoch
            process_bar.show_process(data_num, batch_size=batch_size,
                                     train_loss=mean_train_loss,
                                     train_score=mean_train_score,
                                     test_loss=mean_test_loss,
                                     test_score=mean_test_score)
        print("\n")

    # draw loss figure and score figure
    # especially, loss in train dataset use train_loss not mean_loss in drawing
    if draw:
        # set figure format
        set_fig_display(axes_spines_state=[True] * 4)
        # add new figure and subplot
        fig = plt.figure()
        ax1 = fig.add_subplot(111)
        ax1.plot(range(len(train_loss_list)), train_loss_list,
                 label="train_loss")
        if len(test_loss_list) > 0:
            ax1.plot(range(len(train_loss_list)), test_loss_list,
                     label="test_loss")
        ax1.set_xlabel("epoch num")
        ax1.set_ylabel("loss")

        # ax2
        if evaluate is not None:
            ax2 = ax1.twinx()
            ax2.plot(range(len(train_score_list)), train_score_list, "c-",
                     label="train_score", alpha=0.8)
            if len(test_score_list) > 0:
                ax2.plot(range(len(train_score_list)), test_score_list,
                         "r-", label="test_score", alpha=0.8)
            ax2.set_ylabel("score")
            ax2.set_ylim([0, 1.1])

        if test_iter is None and evaluate is None:
            legend_labels = ["train_loss"]
            legend_loc = [0.75, 0.82]
        elif test_iter is None:
            legend_labels = ["train_loss", "train_score"]
            legend_loc = [0.58, 0.82]
        elif evaluate is None:
            legend_labels = ["train_loss", "test_loss"]
            legend_loc = [0.595, 0.82]
        else:
            legend_labels = ["train_loss", "test_loss",
                             "train_score", "test_score"]
            legend_loc = [0.58, 0.78]
        fig.legend(labels=legend_labels, ncol=2, loc=legend_loc)

        if save_fig:
            if not os.path.exists(save_path):
                os.mkdir(save_path)
            fig_name = "fig" + datetime.now().strftime("%Y%m%d_%H%M%S") + ".jpg"
            plt.savefig(save_path + fig_name, dpi=200)

        plt.show()